In [1]:
%%time
# Import libraries and load monthly stock price data

import numpy as np
import pandas as pd
import math
rf= 0.000195179701757642


##Mean portfolio return = Mean Return * Fractions of Total Capital (Chromosome). 
# Risk-free rate = 0.05 (assumption) 
# Standard deviation of portfolio return = (chromosome * Standard deviation)**2 + Covariance * Respective weights in chromosome.
def mean_portfolio_return(child):
    return np.sum(np.multiply(child,mean_hist_return))
def var_portfolio_return(child):
    part_1 = np.sum(np.multiply(child,sd_hist_return)**2)
    temp_lst=[]
    for i in range(50):
        for j in range(50):
            temp=cov_hist_return.iloc[i][j] * child[i] * child[j]
            temp_lst.append(temp)
    part_2=np.sum(temp_lst)
    return part_1+part_2

# Chromosome: Set of genes i.e. fractions of total capital assigned to each asset
def chromosome(n):
    ''' Generates set of random numbers whose sum is equal to 1
        Input: Number of stocks.
        Output: Array of random numbers'''
    ch = np.random.rand(n)
    return ch/sum(ch)

## Fitness function : The Sharpe ratio, S = (µ − r)/σ
## Maximize return, minimize risk
def fitness_fuction(child):
    ''' This will return the Sharpe ratio for a particular portfolio.
        Input: A child/chromosome (1D Array)
        Output: Sharpe Ratio value (Scalar)'''
    return (mean_portfolio_return(child)-rf)/np.sqrt(var_portfolio_return(child))

def Select_elite_population(population, frac=0.3):
    ''' Select elite population from the total population based on fitness function values.
        Input: Population and fraction of population to be considered as elite.
        Output: Elite population.'''
    population = sorted(population,key = lambda x: fitness_fuction(x),reverse=True)
    percentage_elite_idx = int(np.floor(len(population)* frac))
    return population[:percentage_elite_idx]

def mutation(parent):
    ''' Randomy choosen elements of a chromosome are swapped
        Input: Parent
        Output: Offspring (1D Array)'''
    child=parent.copy()
    n=np.random.choice(range(50),2)
    while (n[0]==n[1]):
        n=np.random.choice(range(50),2)
    child[n[0]],child[n[1]]=child[n[1]],child[n[0]]
    return child

def Heuristic_crossover(parent1,parent2):
    ''' The oﬀsprings are created according to the equation:
            Off_spring A = Best Parent  + β ∗ ( Best Parent − Worst Parent)
            Off_spring B = Worst Parent - β ∗ ( Best Parent − Worst Parent)
                Where β is a random number between 0 and 1.
        Input: 2 Parents
        Output: 2 Children (1d Array)'''
    ff1=fitness_fuction(parent1)
    ff2=fitness_fuction(parent2)
    diff=parent1 - parent2
    beta=np.random.rand()
    if ff1>ff2:
        child1=parent1 + beta * diff
        child2=parent2 - beta * diff
    else:
        child2=parent1 + beta * diff
        child1=parent2 - beta * diff
    return child1,child2

def Arithmetic_crossover(parent1,parent2):
    ''' The oﬀsprings are created according to the equation:
            Off spring A = α ∗ Parent1 + (1 −α) ∗ Parent2
            Off spring B = (1 −α) ∗ Parent1 + α ∗ Parent2
            
                Where α is a random number between 0 and 1.
        Input: 2 Parents
        Output: 2 Children (1d Array)'''
    alpha = np.random.rand()
    child1 = alpha * parent1 + (1-alpha) * parent2
    child2 = (1-alpha) * parent1 + alpha * parent2
    return child1,child2

def next_generation(pop_size,elite,crossover=Arithmetic_crossover):
    ''' Generates new population from elite population with mutation probability as 0.4 and crossover as 0.6. 
        Over the final stages, mutation probability is decreased to 0.1.
        Input: Population Size and elite population.
        Output: Next generation population (2D Array).'''
    new_population=[]
    elite_range=range(len(elite))
    #print(elite_range)
    while len(new_population) < pop_size:
        if len(new_population) > 2*pop_size/3: # In the final stages mutation frequency is decreased.
            mutate_or_crossover = np.random.choice([0, 1], p=[0.6, 0.4])
        else:
            mutate_or_crossover = np.random.choice([0, 1], p=[0.4, 0.6])
        #print(mutate_or_crossover)
        if mutate_or_crossover:
            indx=np.random.choice(elite_range)
            new_population.append(mutation(elite[indx]))
            #print('Mutation!')
        else:
            p1_idx,p2_idx=np.random.choice(elite_range,2)
            c1,c2=crossover(elite[p1_idx],elite[p2_idx])
            chk=0
            for gene in range(50):
                if c1[gene]<0:
                    chk+=1
                else:
                    chk+=0
            if chk>0:
                p1_idx,p2_idx=np.random.choice(elite_range,2)
                c1,c2=crossover(elite[p1_idx],elite[p2_idx])
            
            new_population.extend([c1,c2])
            #print('Crossover!')    
    return new_population


hist_stock_returns = pd.read_csv('Nifty50returnsDec2017-Dec2020.csv')
hist_stock_returns['date'] = pd.to_datetime(hist_stock_returns['date'], format='%d-%m-%Y')
hist_stock_returns.set_index('date',inplace=True)

mean_hist_return=hist_stock_returns.mean()
#print(mean_hist_return)
sd_hist_return=hist_stock_returns.std()
#print(sd_hist_return)
cov_hist_return=hist_stock_returns.cov()
#print(cov_hist_return)
for i in range(50):
    cov_hist_return.iloc[i][i]=0
#print(cov_hist_return)

    
## Determine the Optimum portfolio using Arithmetic Crossover  

n=50 # Number of stocks = 6
pop_size=250 # initial population = 100

# Initial population
population = np.array([chromosome(n) for _ in range(pop_size)])

# Get initial elite population
elite = Select_elite_population(population)

iteration=0 
Expected_returns=10
Expected_risk=15

plotData = {
    "expReturns": [],
    "expRisk": [],
    "SharpeRatio": [] 
}

while (Expected_returns < 30 or Expected_risk > 10) and iteration <= 300:
        
    population = next_generation(250,elite)
    elite = Select_elite_population(population)
    
    Expected_returns=(((1+mean_portfolio_return(elite[0]))**250)-1)*100
    Expected_risk=math.sqrt(var_portfolio_return(elite[0]))*math.sqrt(250)*100
    SR = (Expected_returns-5)/Expected_risk
    
    print('Iteration:',iteration)    
    print('Expected returns of {} with risk of {} and SR = {}\n'.format(round(Expected_returns,ndigits=2),round(Expected_risk,ndigits=2),round(SR,ndigits=2)))
    plotData['expReturns'].extend([Expected_returns])
    plotData['expRisk'].extend([Expected_risk])
    plotData['SharpeRatio'].extend([SR])
    
    iteration+=1

print('\n\n=====================================================\nPortfolio of stocks after all the iterations:\n')
[print(hist_stock_returns.columns[i],':',elite[0][i]) for i in list(range(50))]

print('\nExpected returns of {} with risk of {} and Sharpe Ratio of {}\n'.format(round(Expected_returns,ndigits=2),round(Expected_risk,ndigits=2),round(SR,ndigits=2)))

companies = []
wts = []
for i in list(range(50)):
    companies = companies + [hist_stock_returns.columns[i]]
    wts = wts + [elite[0][i]]

wtDf = pd.DataFrame(columns=['Company','Weight'])
wtDf.Company = companies
wtDf.Weight = wts
wtDf.to_csv('OptimizedWeightsJan2021-Jun2021.csv',index=False)

Iteration: 0
Expected returns of 16.7 with risk of 20.77 and SR = 0.56

Iteration: 1
Expected returns of 17.34 with risk of 20.58 and SR = 0.6

Iteration: 2
Expected returns of 17.25 with risk of 20.48 and SR = 0.6

Iteration: 3
Expected returns of 17.67 with risk of 20.59 and SR = 0.62

Iteration: 4
Expected returns of 17.81 with risk of 20.6 and SR = 0.62

Iteration: 5
Expected returns of 18.44 with risk of 20.48 and SR = 0.66

Iteration: 6
Expected returns of 18.23 with risk of 20.46 and SR = 0.65

Iteration: 7
Expected returns of 18.46 with risk of 20.41 and SR = 0.66

Iteration: 8
Expected returns of 18.37 with risk of 20.41 and SR = 0.66

Iteration: 9
Expected returns of 18.72 with risk of 20.48 and SR = 0.67

Iteration: 10
Expected returns of 19.01 with risk of 20.32 and SR = 0.69

Iteration: 11
Expected returns of 19.31 with risk of 20.5 and SR = 0.7

Iteration: 12
Expected returns of 19.27 with risk of 20.51 and SR = 0.7

Iteration: 13
Expected returns of 19.25 with risk of 20

Iteration: 112
Expected returns of 21.38 with risk of 20.06 and SR = 0.82

Iteration: 113
Expected returns of 21.38 with risk of 20.06 and SR = 0.82

Iteration: 114
Expected returns of 21.37 with risk of 20.06 and SR = 0.82

Iteration: 115
Expected returns of 21.35 with risk of 20.03 and SR = 0.82

Iteration: 116
Expected returns of 21.35 with risk of 20.03 and SR = 0.82

Iteration: 117
Expected returns of 21.4 with risk of 20.08 and SR = 0.82

Iteration: 118
Expected returns of 21.4 with risk of 20.08 and SR = 0.82

Iteration: 119
Expected returns of 21.4 with risk of 20.07 and SR = 0.82

Iteration: 120
Expected returns of 21.35 with risk of 20.02 and SR = 0.82

Iteration: 121
Expected returns of 21.35 with risk of 20.01 and SR = 0.82

Iteration: 122
Expected returns of 21.35 with risk of 20.01 and SR = 0.82

Iteration: 123
Expected returns of 21.36 with risk of 20.03 and SR = 0.82

Iteration: 124
Expected returns of 21.35 with risk of 20.02 and SR = 0.82

Iteration: 125
Expected retu

Iteration: 222
Expected returns of 21.39 with risk of 20.04 and SR = 0.82

Iteration: 223
Expected returns of 21.39 with risk of 20.04 and SR = 0.82

Iteration: 224
Expected returns of 21.39 with risk of 20.04 and SR = 0.82

Iteration: 225
Expected returns of 21.39 with risk of 20.04 and SR = 0.82

Iteration: 226
Expected returns of 21.39 with risk of 20.04 and SR = 0.82

Iteration: 227
Expected returns of 21.39 with risk of 20.03 and SR = 0.82

Iteration: 228
Expected returns of 21.39 with risk of 20.03 and SR = 0.82

Iteration: 229
Expected returns of 21.39 with risk of 20.03 and SR = 0.82

Iteration: 230
Expected returns of 21.38 with risk of 20.03 and SR = 0.82

Iteration: 231
Expected returns of 21.38 with risk of 20.03 and SR = 0.82

Iteration: 232
Expected returns of 21.39 with risk of 20.04 and SR = 0.82

Iteration: 233
Expected returns of 21.39 with risk of 20.04 and SR = 0.82

Iteration: 234
Expected returns of 21.39 with risk of 20.03 and SR = 0.82

Iteration: 235
Expected r

Wall time: 10h 16min 58s


In [2]:
%%time

## Determine the Optimum portfolio using Heuristic Crossover

n=50 # Number of stocks = 6
pop_size=250 # initial population = 100

# Initial population
population = np.array([chromosome(n) for _ in range(pop_size)])

# Get initial elite population
elite = Select_elite_population(population)

iteration=0 
Expected_returns=10
Expected_risk=15

plotData1 = {
    "expReturns": [],
    "expRisk": [],
    "SharpeRatio": [] 
}

while (Expected_returns < 50 or Expected_risk > 5) and iteration <= 100:
    
    print('Iteration:',iteration)
    
    population = next_generation(250,elite,Heuristic_crossover)
    elite = Select_elite_population(population)
    
    Expected_returns=round((((1+mean_portfolio_return(elite[0]))**250)-1)*100,ndigits=1)
    Expected_risk=round(math.sqrt(var_portfolio_return(elite[0]))*math.sqrt(250)*100,ndigits=1)
    SR = (Expected_returns-5)/Expected_risk
    
    print('Expected returns of {} with risk of {} and SR = {}\n'.format(Expected_returns,Expected_risk,SR))
    plotData1['expReturns'].extend([Expected_returns])
    plotData1['expRisk'].extend([Expected_risk])
    plotData1['SharpeRatio'].extend([SR])
    
    iteration+=1

print('\n\n=====================================================\nPortfolio of stocks after all the iterations:\n')
[print(hist_stock_returns.columns[i],':',elite[0][i]) for i in list(range(50))]

print('\nExpected returns of {} with risk of {} and Sharpe Ratio of {}\n'.format(Expected_returns,Expected_risk, SR))

companies = []
wts = []
for i in list(range(50)):
    companies = companies + [hist_stock_returns.columns[i]]
    wts = wts + [elite[0][i]]

wtDf = pd.DataFrame(columns=['Company','Weight'])
wtDf.Company = companies
wtDf.Weight = wts
wtDf.to_csv('OptimizedWeightsJan2021-Jun2021_1.csv',index=False)

Iteration: 0
Expected returns of 19.2 with risk of 21.6 and SR = 0.6574074074074073

Iteration: 1
Expected returns of 20.9 with risk of 22.0 and SR = 0.7227272727272727

Iteration: 2
Expected returns of 21.0 with risk of 22.1 and SR = 0.7239819004524887

Iteration: 3
Expected returns of 25.4 with risk of 24.3 and SR = 0.8395061728395061

Iteration: 4
Expected returns of 25.5 with risk of 22.0 and SR = 0.9318181818181818

Iteration: 5
Expected returns of 29.4 with risk of 21.9 and SR = 1.1141552511415524

Iteration: 6
Expected returns of 35.3 with risk of 26.6 and SR = 1.139097744360902

Iteration: 7
Expected returns of 48.3 with risk of 34.0 and SR = 1.2735294117647058

Iteration: 8
Expected returns of 53.8 with risk of 34.8 and SR = 1.4022988505747127

Iteration: 9
Expected returns of 47.0 with risk of 30.7 and SR = 1.3680781758957654

Iteration: 10
Expected returns of 59.0 with risk of 34.3 and SR = 1.5743440233236152

Iteration: 11
Expected returns of 73.6 with risk of 32.0 and SR =

Expected returns of 5.063579320322797e+290 with risk of 126572.0 and SR = 4.000552507918653e+285

Iteration: 88
Expected returns of 1.1601087832354869e+275 with risk of 107425.4 and SR = 1.0799203756611444e+270

Iteration: 89
Expected returns of 3.3374030399503463e+271 with risk of 102255.5 and SR = 3.263788294957578e+266

Iteration: 90
Expected returns of 6.987151212276958e+270 with risk of 102235.1 and SR = 6.834395635429474e+265

Iteration: 91
Expected returns of 7.249035022046557e+270 with risk of 102286.2 and SR = 7.087011759207554e+265

Iteration: 92
Expected returns of 1.8874263258150197e+270 with risk of 102089.0 and SR = 1.848804793675146e+265

Iteration: 93
Expected returns of 6.942122106628856e+270 with risk of 102040.0 and SR = 6.803334091169007e+265

Iteration: 94
Expected returns of 1.1778812918921689e+269 with risk of 99839.2 and SR = 1.1797783755200051e+264

Iteration: 95
Expected returns of 9.067849608159006e+269 with risk of 101376.9 and SR = 8.94469016921903e+264

It

In [3]:
pd.DataFrame(plotData).to_csv('C1Results.csv')
pd.DataFrame(plotData1).to_csv('C1Results_Heuristic.csv')